In [16]:
!pip install beautifulsoup4

   ---------------------------------------- 147.9/147.9 kB 8.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install pandas numpy scikit-learn nltk streamlit pillow requests

   ---------------------------------------- 8.7/8.7 MB 39.9 MB/s eta 0:00:00
   --------------------------------------- 658.1/658.1 kB 43.2 MB/s eta 0:00:00
   --------------------------------------- 207.3/207.3 kB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 25.1/25.1 MB 38.4 MB/s eta 0:00:00
   ---------------------------------------- 6.9/6.9 MB 40.1 MB/s eta 0:00:00
   --------------------------------------- 241.6/241.6 kB 14.5 MB/s eta 0:00:00
   ---------------------------------------- 82.9/82.9 kB ? eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 87.5/87.5 kB ? eta 0:00:00
   ---------------------------------------- 167.7/167.7 kB ? eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# A.I. Generated Images

In [1]:
!pip install -U diffusers

   ---------------------------------------- 2.7/2.7 MB 34.0 MB/s eta 0:00:00
   ---------------------------------------- 436.4/436.4 kB ? eta 0:00:00
   --------------------------------------- 285.9/285.9 kB 17.2 MB/s eta 0:00:00
   --------------------------------------- 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   --------------------------------------- 161.8/161.8 kB 10.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install accelerate


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   --------------------------------------- 324.4/324.4 kB 20.9 MB/s eta 0:00:00


In [1]:
import torch
from diffusers import FluxPipeline
import time
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# Increase batch size significantly
batch_size = 16

# Increase the number of batches
num_batches = 10

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.float16)
pipe = pipe.to(device)

# Enable attention slicing to reduce memory usage if needed
pipe.enable_attention_slicing()

prompts = [
    "A futuristic city with flying cars and neon lights",
    "A serene mountain landscape with a clear lake reflection",
    "An underwater scene with colorful coral reefs and exotic fish",
    "A bustling marketplace in a medieval fantasy town",
    "A spacecraft landing on an alien planet with two moons",
    "A cozy library filled with ancient books and magical artifacts",
    "A steampunk-inspired train station with clockwork mechanisms",
    "A enchanted forest with glowing plants and mystical creatures",
    "A cyberpunk street scene with holographic advertisements",
    "A peaceful zen garden with cherry blossoms and a koi pond",
    "An epic battle scene between dragons and knights",
    "A whimsical candy land with chocolate rivers and lollipop trees",
    "A post-apocalyptic cityscape reclaimed by nature",
    "A grand ballroom with elaborate chandeliers and dancers in period costumes",
    "A mad scientist's laboratory filled with bubbling potions and strange contraptions",
    "A robot repair shop with androids in various states of assembly"
]

start_time = time.time()

all_images = []

for batch in tqdm(range(num_batches), desc="Generating batches"):
    images = pipe(
        prompts,
        guidance_scale=7.5,  # Increase guidance scale for more detailed images
        num_inference_steps=50,  # Increase number of steps for higher quality
        max_sequence_length=512,  # Increase sequence length for more complex scenes
        generator=torch.Generator(device=device).manual_seed(batch)
    ).images
    all_images.extend(images)

end_time = time.time()

print(f"Time taken to generate {batch_size * num_batches} images: {end_time - start_time:.2f} seconds")

for i, image in enumerate(all_images):
    image.save(f"flux_high_utilization_{i}.png")

print(f"Total images generated: {len(all_images)}")

# Force CUDA to finish all operations and wait
torch.cuda.synchronize()
time.sleep(5)

print("Script completed. Check Task Manager for final GPU utilization.")

Using device: cuda


Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:  36%|###6      | 1.65G/4.53G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:  76%|#######5  | 3.79G/4.99G [00:00<?, ?B/s]

(…)pytorch_model-00001-of-00003.safetensors:  16%|#5        | 1.55G/9.96G [00:00<?, ?B/s]

(…)pytorch_model-00003-of-00003.safetensors:  66%|######5   | 2.54G/3.87G [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00003.safetensors:  19%|#8        | 1.89G/9.95G [00:00<?, ?B/s]

c:\Users\joshf\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joshf\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# load data
df = pd.read_csv('../data/rawdata.csv')

In [15]:
import torch
from diffusers import StableDiffusionPipeline
import pandas as pd
import os
from tqdm import tqdm
import time
import random

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# Load the Etsy product data
df = pd.read_csv('../data/rawdata.csv')

# Configure the pipeline
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

# Set up batch processing
batch_size = 4
num_batches = (len(df) + batch_size - 1) // batch_size

# Ensure the images directory exists
os.makedirs('../data/images', exist_ok=True)

# Function to generate e-commerce focused product prompts
def generate_prompt(product_name):
    styles = [
        "professional product photography", "studio product shot", "e-commerce style image",
        "high-quality product display", "artistic product arrangement", "minimalist product showcase"
    ]
    lighting = [
        "soft diffused lighting", "bright even illumination", "subtle shadow highlighting",
        "crisp studio lighting", "dramatic product highlighting"
    ]
    backgrounds = [
        "pure white backdrop", "light grey seamless background", "subtle gradient background",
        "clean minimal background", "soft focus background"
    ]
    details = [
        "sharp focus on product details", "floating product display",
        "product-only composition", "full product shot"
    ]
    
    style = random.choice(styles)
    light = random.choice(lighting)
    background = random.choice(backgrounds)
    detail = random.choice(details)
    
    prefix = f"Professional e-commerce photograph of {product_name},"
    suffix = f"{style}, {light}, {background}, {detail}, no people, for online selling"
    return f"{prefix} {suffix}"

# Process batches
start_time = time.time()

for batch in tqdm(range(num_batches), desc="Generating images"):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, len(df))
    
    batch_products = df['product_name'].iloc[start_idx:end_idx]
    batch_prompts = [generate_prompt(name) for name in batch_products]
    
    images = pipe(
        batch_prompts,
        guidance_scale=7.5,
        num_inference_steps=30,  # Increased to 50 as requested
        width=512,
        height=512,
        generator=torch.Generator(device=device).manual_seed(batch)
    ).images
    
    for i, (image, product_name) in enumerate(zip(images, batch_products)):
        safe_name = ''.join(c if c.isalnum() else '_' for c in product_name)[:50]
        image_path = f"../data/images/{safe_name}.png"
        image.save(image_path)
        df.loc[start_idx + i, 'image_path'] = image_path

end_time = time.time()
print(f"Time taken to generate {len(df)} images: {end_time - start_time:.2f} seconds")

# Save the updated dataframe with image paths
df.to_csv('../data/rawdata_with_images.csv', index=False)

print("Image generation complete. Updated CSV saved with image paths.")

# Force CUDA to finish all operations and wait
torch.cuda.synchronize()
time.sleep(5)

print("Script completed. Check the '../data/images' folder for generated images.")

Using device: cuda


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\joshf\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating images:   0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Generating images:   0%|          | 1/250 [00:20<1:25:41, 20.65s/it]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Generating images: 100%|█████████▉| 249/250 [1:31:27<00:22, 22.08s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

Generating images: 100%|██████████| 250/250 [1:31:49<00:00, 22.04s/it]


Time taken to generate 1000 images: 5509.98 seconds
Image generation complete. Updated CSV saved with image paths.
Script completed. Check the '../data/images' folder for generated images.


## Scrape images

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse

def scrape_and_save_image(url, save_path):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        img_tag = soup.find('img', {'data-index': '0'})
        
        if img_tag and 'src' in img_tag.attrs:
            img_url = img_tag['src']
            img_response = requests.get(img_url, headers=headers)
            
            if img_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(img_response.content)
                return True
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
    return False

# Load the data
df = pd.read_csv('../data/rawdata.csv')

# Create images directory if it doesn't exist
os.makedirs('../data/images', exist_ok=True)

# Scrape and save images
for index, row in df.iterrows():
    product_id = urlparse(row['product_link']).path.split('/')[-1]
    image_path = f'../data/images/{product_id}.jpg'
    
    if not os.path.exists(image_path):
        success = scrape_and_save_image(row['product_link'], image_path)
        if success:
            print(f"Saved image for product {product_id}")
        else:
            print(f"Failed to save image for product {product_id}")

# Update DataFrame with local image paths
df['local_image_path'] = df['product_link'].apply(lambda x: f'../data/images/{urlparse(x).path.split("/")[-1]}.jpg')

# Save updated DataFrame
df.to_csv('../data/rawdata_with_images.csv', index=False)

print("Image scraping and CSV update complete.")

Failed to save image for product wooden-washer-and-dryer-top-laundry
Failed to save image for product the-complete-nursing-school-bundle-o
Failed to save image for product personalized-name-necklace-name-logo
Failed to save image for product marble-wood-charcuterie-board-wedding
Failed to save image for product madam-president-kamala-harris-and-tim
Failed to save image for product personalized-mama-sweatshirt-with-kid
Failed to save image for product personalized-hand-embroidered-name-baby
Failed to save image for product large-wood-name-sign-nursery-name-sign
Failed to save image for product personalized-name-necklace-by
Failed to save image for product personalized-signature-necklace-custom
Failed to save image for product wooden-world-map-enjoy-the-wood-home
Failed to save image for product x10-custom-wish-spell-make-a-wish-wish
Failed to save image for product custom-name-necklace-18k-gold-plated
Failed to save image for product ateez-mito-crochet-lightiny-cover
Failed to save imag

KeyboardInterrupt: 

# Data preprocessing

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
df = pd.read_csv('../data/rawdata.csv')

# Combine all tag columns into a single text feature
tag_columns = [f'tag_{i}' for i in range(1, 14)]  # Create a list of tag column names
df['text_features'] = df[tag_columns].fillna('').agg(' '.join, axis=1)

# Create TF-IDF vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text_features'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get recommendations based on similarity
def get_recommendations(idx, cosine_sim=cosine_sim):
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:7]  # Get top 6 similar items
    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices]

# NLP Tag Matching

In [12]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower().replace('_', ' '))
    return list(synonyms)

def match_tags(recipient, occasion):
    recipient_synonyms = get_synonyms(recipient)
    occasion_synonyms = get_synonyms(occasion)
    
    matching_products = df[df['text_features'].apply(lambda x: any(tag in x.lower() for tag in recipient_synonyms + occasion_synonyms))]
    
    return matching_products

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joshf\AppData\Roaming\nltk_data...


# Recommendation engine

In [13]:
def get_recommendations_by_category(matching_products):
    recommendations = {
        'Trending': matching_products.sort_values('listing_age', ascending=True).iloc[0],
        'Most Favorited': matching_products.sort_values('favorites', ascending=False).iloc[0],
        'Most Popular': matching_products.sort_values('est_mo_sales', ascending=False).iloc[0],
        'Quirky': matching_products.sample().iloc[0],
        'Best Seller': matching_products.sort_values('est_total_sales', ascending=False).iloc[0],
        'Highest Rated': matching_products.sort_values('avg_reviews', ascending=False).iloc[0]
    }
    return recommendations

def surprise_me(recipient, occasion):
    matching_products = match_tags(recipient, occasion)
    recommendations = get_recommendations_by_category(matching_products)
    return recommendations

In [14]:
import streamlit as st
import requests
from PIL import Image
from io import BytesIO

def load_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

st.title("Etsy Gift Finder")

recipient_options = ["Girlfriend", "Mom", "Dad", "Grandfather", "Grandmother", "Friend", "Coworker"]
occasion_options = ["Birthday", "Wedding", "Valentine's Day", "Christmas", "Anniversary", "Graduation"]

recipient = st.selectbox("Who is the gift for?", recipient_options)
occasion = st.selectbox("What's the occasion?", occasion_options)

if st.button("Surprise Me"):
    recommendations = surprise_me(recipient, occasion)
    
    for category, product in recommendations.items():
        st.subheader(category)
        st.write(product['product_name'])
        st.write(f"Price: ${product['price']}")
        st.write(f"Shop: {product['shop_name']}")
        
        image = load_image(product['product_link'])
        st.image(image, use_column_width=True)
        
        if st.button(f"View on Etsy", key=category):
            st.markdown(f"[View on Etsy]({product['product_link']})")
        
        st.write("---")

2024-09-20 09:33:30.420 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.011 
  command:

    streamlit run c:\Users\joshf\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-20 09:33:31.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.014 Session state does not function when running a script without `streamlit run`
2024-09-20 09:33:31.015 Thread 'MainThrea